In [1]:
# to add the spiking model codebase to the path
import sys
sys.path.append('..')

%matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib.animation import FuncAnimation
# plt.style.use('seaborn-pastel')

from brian2 import pA, Hz, second, ms
import numpy as np

from results import Results
from plot import (MidpointNormalize, weight_matrix, remove_axes_less, prune_figure, prune_figure_less, prune_figure_more,
                  plot_spike_raster, plot_firing_rate, plot_voltage_detail)

from generate import weight_matrix_from_flat_inds_weights, adjacency_matrix_from_flat_inds, adjacency_indices_within
import pandas as pd
from sklearn.neighbors import KernelDensity
def cv(a):
    return a.std() / a.mean()

import os

from stats import moving_average

from tqdm import tqdm

Using matplotlib backend: Qt5Agg


INFO       Cache size for target "cython": 8765 MB.
You can call "clear_cache('cython')" to delete all files from the cache or manually delete files in the "C:\Users\mikejseay\.cython\brian_extensions" directory. [brian2]


In [13]:
# what i want to understand is how the matrix of proposed dW for one matrix, say EI, differs for the two rules
# perhaps to be most simple we should take a single converged final matrix and calculate a single set of proposed weight changes

# let's use seed 2

# in the end game it seems like regardless of whether the presynaptic multiplier is included, the proposed changes to wEE and wEI
# are ridiculous.. they keep diverging because the inhibitory FR stays above the set point

In [35]:
targetPath = 'C:/Users/mikejseay/Documents/BrianResults/cross-homeo-pre-outer-homeo-explodeDealTest'
targetFile = 'classicJercog_2000_0p25_cross-homeo-pre-outer-homeo_resumePrior_seed2explodeDealTestLowAlpha_2022-02-03-08-02-51_results'

# targetPath = 'C:/Users/mikejseay/Documents/BrianResults/cross-homeo-pre-scalar-homeo-explodeDealTest'
# targetFile = 'classicJercog_2000_0p25_cross-homeo-pre-scalar-homeo_seed2_explodeDealTestLowAlpha_2022-02-10-07-30-29_results'

In [ ]:
R = Results()
R.init_from_file(targetFile, targetPath)
p = R.p

In [61]:
setUpFRExc = p['setUpFRExc'] / Hz
setUpFRInh = p['setUpFRInh'] / Hz

In [48]:
movAvgUpFRExcUnits = R.trialUpFRExcUnits[-1, :]
movAvgUpFRInhUnits = R.trialUpFRInhUnits[-1, :]

In [63]:
movAvgUpFRInhUnits.mean()

14.196691176470587

In [60]:
dwEIS = setUpFRInh - movAvgUpFRInhUnits.mean()  # scalar

In [67]:
dwEIS

-0.19669117647058698

In [58]:
aEE = adjacency_matrix_from_flat_inds(p['nExc'], p['nExc'], R.preEE, R.posEE)
aEI = adjacency_matrix_from_flat_inds(p['nInh'], p['nExc'], R.preEI, R.posEI)
aIE = adjacency_matrix_from_flat_inds(p['nExc'], p['nInh'], R.preIE, R.posIE)
aII = adjacency_matrix_from_flat_inds(p['nInh'], p['nInh'], R.preII, R.posII)

nIncomingExcOntoEachExc = aEE.sum(0)
nIncomingInhOntoEachExc = aEI.sum(0)
nIncomingExcOntoEachInh = aIE.sum(0)
nIncomingInhOntoEachInh = aII.sum(0)

In [59]:
movAvgUpFRInhUnitsPreToPostExc = np.matmul(movAvgUpFRInhUnits, aEI) / nIncomingInhOntoEachExc
movAvgUpFRExcUnitsPreToPostInh = np.matmul(movAvgUpFRExcUnits, aIE) / nIncomingExcOntoEachInh

In [62]:
movAvgUpFRInhUnitsPreToPostExc.mean()

14.188997053862186

In [64]:
dwEIO = setUpFRInh - movAvgUpFRInhUnitsPreToPostExc

In [66]:
dwEIO.mean()

-0.18899705386218613

In [ ]:
# compare seed 2...
# around trial 550 outer has started to completely fail, while scalar is going just great


In [68]:
targetPath1 = 'C:/Users/mikejseay/Documents/BrianResults/cross-homeo-pre-outer-explodeDealTest/'
targetPath2 = 'C:/Users/mikejseay/Documents/BrianResults/cross-homeo-pre-scalar-explodeDealTest/'

In [69]:
targetFile1 = 'classicJercog_2000_0p25_cross-homeo-pre-outer_seed2_explodeDealTest_2022-02-10-07-29-19_results'
targetFile2 = 'classicJercog_2000_0p25_cross-homeo-pre-scalar_seed2_explodeDealTestLowAlpha_2022-02-09-07-28-45_results'

In [70]:
R1 = Results()
R1.init_from_file(targetFile1, targetPath1)

R2 = Results()
R2.init_from_file(targetFile2, targetPath2)

In [7]:
nTrials = R1.p['nTrials']
nRules = 2

In [75]:
frCorrEI.shape

(2000, 2000)

In [76]:
R1.trialUpFRExcUnits[startTrialInd:endTrialInd].T.shape

(1600, 550)

In [77]:
R1.trialUpFRInhUnits[startTrialInd:endTrialInd].T.shape

(400, 550)

In [83]:
R1.trialdwEEUnits

(6000, 1600)

In [84]:
# let's just try to compare the individual unit FRs over the course of training

In [ ]:
from analysis import weight_matrix

In [94]:
# plotting the FR for each unit for each trial as an implot

# startTrialInd = 800
# endTrialInd = 1400

startTrialInd = 0
endTrialInd = -1

# maxFRExc = 150
# maxFRInh = 150

useCmap = plt.cm.Reds
f, ax = plt.subplots(2, 2, sharex=True, sharey=False)

useDataExc1 = R1.trialUpFRExcUnits[startTrialInd:endTrialInd, :].T
useDataInh1 = R1.trialUpFRInhUnits[startTrialInd:endTrialInd, :].T
useDataExc2 = R2.trialUpFRExcUnits[startTrialInd:endTrialInd, :].T
useDataInh2 = R2.trialUpFRInhUnits[startTrialInd:endTrialInd, :].T

# useDataExc[useDataExc > maxFRExc] = np.nan
# useDataInh[useDataInh > maxFRInh] = np.nan

i00 = ax[0, 0].imshow(useDataExc1, cmap=plt.cm.Greens, aspect='auto', interpolation='none')
i10 = ax[1, 0].imshow(useDataInh1, cmap=plt.cm.Reds, aspect='auto', interpolation='none')

i01 = ax[0, 1].imshow(useDataExc2, cmap=plt.cm.Greens, aspect='auto', interpolation='none')
i11 = ax[1, 1].imshow(useDataInh2, cmap=plt.cm.Reds, aspect='auto', interpolation='none')

cb00 = plt.colorbar(i0, ax=ax[0, 0])
cb10 = plt.colorbar(i1, ax=ax[1, 0])

cb01 = plt.colorbar(i0, ax=ax[0, 1])
cb11 = plt.colorbar(i1, ax=ax[1, 1])

In [87]:
# looks like there is a bit of "whack-a-mole" going on with the individual unit rates...
# some units have a high rate for part of the training, then fade and then other take their place
# whereas with the scalar version they tend to either stay high or low
# let's visualize the variance in FR across units as a moving average

In [88]:
R1.trialUpFRExcUnits.shape

(6000, 1600)

In [95]:
f, ax = plt.subplots()

ax.plot(R1.trialUpFRExcUnits.std(1), label='E-outer')
ax.plot(R1.trialUpFRInhUnits.std(1), label='I-outer')
ax.plot(R2.trialUpFRExcUnits.std(1), label='E-scalar')
ax.plot(R2.trialUpFRInhUnits.std(1), label='I-scalar')

ax.legend()

In [ ]:
# indeed, over the course of training, the outer rule appears to be boosting the variance of the FRs across units
# whereas with the scalar they reach asymptotic values
# so a key thing to look at will be how the computed dW values alter the E/I balance to individual units on a given trial
# if the E/I balance shifts in different directions across units, their FR should change and spread apart, increasing variance
# and this will only become more of a problem over time,

In [96]:
plt.close('all')

In [82]:
# just for reference, let's check the correlations

startTrialInd = 0
endTrialInd = -1

frCorrEE = np.corrcoef(R1.trialUpFRExcUnits[startTrialInd:endTrialInd].T)
frCorrEI = np.corrcoef(R1.trialUpFRExcUnits[startTrialInd:endTrialInd].T, R1.trialUpFRInhUnits[startTrialInd:endTrialInd].T)
frCorrII = np.corrcoef(R1.trialUpFRInhUnits[startTrialInd:endTrialInd].T)

frCorrEE[np.triu_indices(frCorrEE.shape[0])] = np.nan
frCorrEI[np.triu_indices(frCorrEI.shape[0])] = np.nan
frCorrII[np.triu_indices(frCorrII.shape[0])] = np.nan

frCorrEEAnti = frCorrEE.copy()
frCorrEIAnti = frCorrEI.copy()
frCorrIIAnti = frCorrII.copy()

frCorrEEAnti[R1.preEE, R1.posEE] = np.nan
frCorrEIAnti[R1.preEI, R1.posEI] = np.nan
frCorrEIAnti[R1.preIE, R1.posIE] = np.nan
frCorrIIAnti[R1.preII, R1.posII] = np.nan

print('R1: outer')
print(np.nanmean(frCorrEE))  # average inter-unit correlation across trials (E to E)
print('*', np.nanmean(frCorrEE[R1.preEE, R1.posEE])) # for only connect units
print(np.nanmean(frCorrEEAnti))  # for only unconnected units

print(np.nanmean(frCorrEI)) # average inter-unit correlation across trials (E to I)
print('*', np.nanmean(frCorrEI[R1.preEI, R1.posEI])) # correlation between E and I units for which I --> E
print('*', np.nanmean(frCorrEI[R1.preIE, R1.posIE])) # correlation between E and I units for which E --> I
print(np.nanmean(frCorrEIAnti))  # for only unconnected units

print(np.nanmean(frCorrII))  # average inter-unit correlation across trials (I to I)
print('*', np.nanmean(frCorrII[R1.preII, R1.posII])) # for only connect units
print(np.nanmean(frCorrIIAnti))  # for only unconnected units

frCorrEE = np.corrcoef(R2.trialUpFRExcUnits[startTrialInd:endTrialInd].T)
frCorrEI = np.corrcoef(R2.trialUpFRExcUnits[startTrialInd:endTrialInd].T, R2.trialUpFRInhUnits[startTrialInd:endTrialInd].T)
frCorrII = np.corrcoef(R2.trialUpFRInhUnits[startTrialInd:endTrialInd].T)

frCorrEE[np.triu_indices(frCorrEE.shape[0])] = np.nan
frCorrEI[np.triu_indices(frCorrEI.shape[0])] = np.nan
frCorrII[np.triu_indices(frCorrII.shape[0])] = np.nan

frCorrEEAnti = frCorrEE.copy()
frCorrEIAnti = frCorrEI.copy()
frCorrIIAnti = frCorrII.copy()

frCorrEEAnti[R2.preEE, R2.posEE] = np.nan
frCorrEIAnti[R2.preEI, R2.posEI] = np.nan
frCorrEIAnti[R2.preIE, R2.posIE] = np.nan
frCorrIIAnti[R2.preII, R2.posII] = np.nan

print('R2: scalar')
print(np.nanmean(frCorrEE))
print('*', np.nanmean(frCorrEE[R2.preEE, R2.posEE]))
print(np.nanmean(frCorrEEAnti))

print(np.nanmean(frCorrEI))
print('*', np.nanmean(frCorrEI[R2.preEI, R2.posEI]))
print('*', np.nanmean(frCorrEI[R2.preIE, R2.posIE]))
print(np.nanmean(frCorrEIAnti))

print(np.nanmean(frCorrII))
print('*', np.nanmean(frCorrII[R2.preII, R2.posII]))
print(np.nanmean(frCorrIIAnti))

R1: outer
0.28133488068379847
* 0.28420836516923526
0.2803752746136089
0.23023272052572077
* 0.25075490265988026
* 0.26948269775518724
0.22709786282769243
0.08303565008657615
* 0.07550921675137845
0.08554345525961114
R2: scalar
0.14315690579864632
* 0.15242798757242146
0.14006080891960343
0.07036389038471792
* 0.1292407598295128
* 0.13649879856056057
0.06486852839082326
0.04001453424250512
* 4.269631976194476e-05
0.053333137816602774


In [ ]:
trialUpFRExc = np.stack((R1.trialUpFRExc, R2.trialUpFRExc))
trialUpFRInh = np.stack((R1.trialUpFRInh, R2.trialUpFRInh))

In [ ]:
for ruleInd in range(nRules):
    for trialInd in range(nTrials):
        
    